In [ ]:
import colbert

In [ ]:
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

## Indexing

In [ ]:
nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 300 # truncate passages at 300 tokens
max_id = 10000

index_name = f'Mimic.{nbits}bits'

## Data Clean

In [ ]:
import os
import json
import random
import time
import re
import pickle
import traceback
from tqdm import tqdm
import pandas as pd

In [ ]:
queries = [
    "Identify the sentence discussing stage.",
    "Identify the sentence discussing histology",
    "Identify the sentence discussing tumor size",
    "Identify the sentence discussing recurrence"
]

In [ ]:
## Functions to process the data
def DataClean(data,labels_path,data_path=None,delete=None):

    print('\n\nImporting and filtering database...')

    if data is not None and not data.empty:
        notes = data
    else:
        notes = pd.read_csv(data_path)

    ints_str = '0123456789-#[]' # characters that usually main categories don't start with

    print('\n\nSplitting each note into sections:\n\n')

    notes_sections = {}

    for note_index in tqdm(range(notes.shape[0])):
        note = notes['text'][note_index].replace('\n\n\n\n','\n').replace('\n\n\n','\n').replace('     ','\n')
        paragraphs = note.split('\n')

        subsections, new_section = [], ' '
        for p in paragraphs:
            line = p.strip()
            if len(line)>0 and ':' in line and not (line[line.find(':')-1] in ints_str) and not(line[0] in ints_str):
                subsections.append([new_section.strip()])
                new_section = p + ' '
            else:
                new_section += p + ' '
        subsections.append([new_section])
        subsections.pop(0)

        note_sect_tit,note_sect_par = [],[]
        for sect in subsections:
            note_sect_tit += [str(*sect)[0:str(*sect).find(':')]]
            note_sect_par += [str(*sect)[str(*sect).find(':')+1:].strip()]
        note_df = pd.DataFrame({'title':note_sect_tit,'category':'','text':note_sect_par, 'label':''})
        notes_sections[notes['note_id'][note_index]] = note_df

    f = open(labels_path, 'r')
    obj_label = f.readlines()
    obj_label_dict = {}
    i = 0
    for s in obj_label:
        i += 1
        if '/' in s:
            buffer = s.strip('\n').lower().split('/')
            for item in buffer:
                obj_label_dict[item] = i
        else:
            obj_label_dict[s.strip('\n').lower()] = i
    f.close()

    for key in tqdm(list(notes_sections.keys())):
        buffer = 'begin_title'
        t = list(notes_sections[key]['title'])
        for idx in range(len(t)):
            for item in list(obj_label_dict.keys()):
                if item in t[idx].lower() and len(t[idx].lower())>2:
                    buffer = item
                    notes_sections[key]['category'][idx] = buffer
                    notes_sections[key]['label'][idx] = obj_label_dict[buffer]
                    break
            notes_sections[key]['category'][idx] = buffer
            notes_sections[key]['label'][idx] = obj_label_dict[buffer]

    notes_sections_output = {}
    row_id  = notes_sections.keys()
    for key in tqdm(row_id):
        buffer = ''
        note_sect_tit, note_sect_par, note_sect_lab = [], [], []
        for i in range(len(notes_sections[key]['category'])):
            if buffer != notes_sections[key]['category'][i]:
                buffer = notes_sections[key]['category'][i]
                note_sect_tit.append(buffer)
                note_sect_lab.append(notes_sections[key]['title'][i])
                note_sect_par.append(notes_sections[key]['text'][i])
                # if buffer == 'followup instruction' or buffer == 'follow up' or buffer == 'follow-up':
                #     break
            else:
                note_sect_par[-1] = note_sect_par[-1] + ' ' + notes_sections[key]['title'][i] + ' ' + notes_sections[key]['text'][i]
        note_df = pd.DataFrame({'title': note_sect_tit, 'text': note_sect_par, 'label': note_sect_lab})
        notes_sections_output[key] = note_df


    notes_sections = notes_sections_output

    if delete != None:
        for key,value in notes_sections.items():
            notes_sections[key] = notes_sections[key][~notes_sections[key]['label'].isin(delete)]

    return notes_sections

In [ ]:
df  = pd.read_csv("/content/malignant_neoplasm_updated_first_100_rows.csv").iloc[:10,]


In [ ]:
NotesSections = DataClean(data=df,
                          labels_path="/content/labels.txt",
                          delete=['Name','Admission Date','Discharge Date','Date of Birth','Followup Instructions'])



Importing and filtering database...


Splitting each note into sections:




100%|██████████| 10/10 [00:00<00:00, 1390.73it/s]


## Index & Search

In [ ]:
import re

In [ ]:
checkpoint = 'colbert-ir/colbertv2.0'

Data = {}

for key,value in NotesSections.items():

  Data[key] = {}

  Strings = NotesSections[key]['text'].tolist()

  ## Delete some useless infomation
  paragraphs = [s for s in Strings if s != ""]

  collection = []

  for para in paragraphs:

    Sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z]\.)(?<![A-Z][a-z]\.)(?<! [a-z]\.)(?<![A-Z][a-z][a-z]\.)(?<=\.|\?|\!)\"*\s*\s*(?:\W*)(?<![A-Z])', para)

    collection = collection + [s for s in Sentences if len(s) > 10]

  Data[key]['content'] = ''.join(collection)

  Data[key]['qas'] = []

  with Run().context(RunConfig(nranks=1,rank=1, experiment='notebook')):  # nranks specifies the number of GPUs to use

      config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits, kmeans_niters=4) # kmeans_niters specifies the number of iterations of k-means clustering; 4 is a good and fast default.
                                                                                  # Consider larger numbers for small datasets.
      indexer = Indexer(checkpoint=checkpoint, config=config)
      indexer.index(name=index_name, collection=collection, overwrite=True)

  with Run().context(RunConfig(experiment='notebook')):

    searcher = Searcher(index=index_name, collection=collection)

  for query in queries:

    qas = {}

    print(f"#> {query}")

    qas[query] = []

    # Find the top-k passages for this query
    results = searcher.search(query, k=5)

    # Print out the top-k retrieved passages
    for passage_id, passage_rank, passage_score in zip(*results):
        print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")
        qas[query].append(searcher.collection[passage_id])

    Data[key]['qas'].append(qas)




[Jul 05, 12:17:01] #> Note: Output directory /content/experiments/notebook/indexes/Mimic.2bits already exists


[Jul 05, 12:17:01] #> Will delete 10 files already at /content/experiments/notebook/indexes/Mimic.2bits in 20 seconds...
#> Joined...
[Jul 05, 12:17:36] #> Loading codec...
[Jul 05, 12:17:36] #> Loading IVF...
[Jul 05, 12:17:36] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 5698.78it/s]

[Jul 05, 12:17:36] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 1150.39it/s]

#> Identify the sentence discussing stage.

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . Identify the sentence discussing stage., 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  6709,  1996,  6251, 10537,  2754,  1012,   102,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])

	 [1] 		 8.1 		 No separate primary lesion identified.
	 [2] 		 7.7 		 No pelvic masses are identified.
	 [3] 		 7.0 		 Bilateral symmetrical sclerosis is identified on the iliac side of the sacroiliac joints, consistent with osteitis condensans ilii.
	 [4] 		 6.6 		 Assess for extent of lesions.
	 [5] 		 6.2 		 Calcification is noted within the right pleural cavity (sequ

...

#> Starting...
#> Joined...
[Jul 05, 12:22:53] #> Loading codec...
[Jul 05, 12:22:53] #> Loading IVF...
[Jul 05, 12:22:53] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 5275.85it/s]

[Jul 05, 12:22:53] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 1072.16it/s]

#> Identify the sentence discussing stage.

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . Identify the sentence discussing stage., 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  6709,  1996,  6251, 10537,  2754,  1012,   102,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])

	 [1] 		 6.4 		 There is a heterogeneous 3.
	 [2] 		 6.3 		 BONE WINDOWS No suspicious lytic or sclerotic osseous lesion is identified.
	 [3] 		 6.0 		 The opacified stomach and intra-abdominal loops of bowel are unremarkable.
	 [4] 		 4.8 		 Two calcified densities within the left buttock are noted, likely injection granulomas.
	 [5] 		 4.7 		 Airways are patent to the s

In [ ]:
file_path = 'data.json'

with open(file_path, 'w') as file:
    json.dump(Data, file,indent=4)